In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.optimize import minimize_scalar
from scipy.stats import linregress
import seaborn as sns

In [ ]:
#Load the data
#x_position data pulled from output files located in "GL60_output" and "Ant60_output"

df = pd.read_csv('elmer-model-output.csv')
df

In [ ]:
#Split the dataset into Greenland and Antarctica

greenland = df[df['Type'] == 'greenland']
antarctica = df[df['Type'] == 'antarctic']

In [ ]:
#Sort the data so that it plots nicely

greenland = greenland.sort_values(by = ['k1'])
antarctica = antarctica.sort_values(by = ['k_ratio'])

In [ ]:
#Predict the bifurcation points with the energy minimization model
#based on dimensions and parameters for Elmer/Ice model

###GREENLAND

x0 = 1500 #meters
y0 = 0

x1 = (-2500 - 767.949) / 2 #meters
y1 = (14330.127 + 15330.127) / 2

x2 = (3767.949 + 5500) / 2 #meters
y2 = (15330.127 + 14330.127) / 2

w_0 = 3000 #meters
w_1 = 2000
w_2 = 2000

h_0 = 500 #meters
h_1 = 500 
h_2 = 500 

u_0 = 175 #meters per year

Q_0 = u_0 * h_0 * w_0 #cubic meters per year
Q_1 = 0.5 * Q_0
Q_2 = 0.5 * Q_0

#u_0 = Q_0 / (h_0 * w_0)
u_1 = Q_1 / (h_1 * w_1) #meters per year
u_2 = Q_2 / (h_2 * w_2)

#Basal friction parameter: Bifurcation point is very sensitive to this
b0 = 1e-5
b1 = 1e-5
b2 = 1e-5

mList = np.sort(np.array([1.5e-2, 1.14e-2, 1.65e-3, 2.17e-3, 2.87e-3, 3.78e-3, 5e-3, 6.56e-3, 8.64e-3]))
xListG = []

for m in mList:#:np.linspace(0.2 * 5e-3, 3 * 5e-3, 100):#mList:
    #Shear margin strength parameter:
    m0 = 5e-3 #5e-3 * (60**2*24*365)**(5/3)
    m1 = m #2.5e-3 * (60**2*24*365)**(5/3)
    m2 = 5e-3 #5e-3 * (60**2*24*365)**(5/3)

    A = (5.15e-17) #[Pa^-3][y^-1] warm ice

    #Calculate dissipations
    D0 = (b0 * (Q_0)**2 / (w_0 * h_0**2) + 
                m0 * ((Q_0)**(4/3) * h_0**(2/3) / (A**(1/3) * w_0**(8/3))))
    D1 = (b1 * (Q_1)**2 / (w_1 * h_1**2) + 
                m1 * ((Q_1)**(4/3) * h_1**(2/3) / (A**(1/3) * w_1**(8/3))))
    D2 = (b2 * (Q_2)**2 / (w_2 * h_2**2) + 
                m2 * ((Q_2)**(4/3) * h_2**(2/3) / (A**(1/3) * w_2**(8/3))))


    #Find the minimum dissipation point
    min_r = float('inf')
    min_x = 0.

    def dissipation(x):
        yb = 10000.
        r = D0 * np.sqrt((x - x0)**2 + (yb - y0)**2) + D1 * np.sqrt((x - x1)**2 + (yb - y1)**2) + D2 * np.sqrt((x - x2)**2 + (yb - y2)**2)
        return r

    out = minimize_scalar(dissipation, bounds = (0, 3000), method = 'bounded')
    min_x = out.x
    xListG.append(min_x)

In [ ]:
#Predict the bifurcation points with the energy minimization model
#based on dimensions and parameters for Elmer/Ice model


###ANTARCTICA

x0 = 15000 #meters
y0 = 0

x1 = (-25000 - 7679.49) / 2 #meters
y1 = (143301.27 + 153301.27) / 2

x2 = (37679.49 + 55000) / 2 #meters
y2 = (153301.27 + 143301.27) / 2

w_0 = 30000 #meters
w_1 = 20000
w_2 = 20000

h_0 = 500 #meters
h_1 = 500 
h_2 = 500 

u_0 = 1000 #meters per year

Q_0 = u_0 * h_0 * w_0 #cubic meters per year
Q_1 = 0.5 * Q_0
Q_2 = 0.5 * Q_0

#u_0 = Q_0 / (h_0 * w_0)
u_1 = Q_1 / (h_1 * w_1) #meters per year
u_2 = Q_2 / (h_2 * w_2)

#Basal friction parameter: Bifurcation point is very sensitive to this

m0 = 0#5e-2 
m1 = 0#5e-2 
m2 = 0#5e-2
bList = np.sort(np.array([1.2e-4, 1.33e-5, 1.75e-5, 2.31e-5, 3.04e-5, 4e-5, 5.26e-5, 6.93e-5, 9.12e-5]))
xListA = []

for b in bList:#np.linspace(0.2 * 4e-5, 4 * 4e-5, 100):#bList:
    #Shear margin strength parameter:
    b0 = 4e-5
    b1 = b
    b2 = 4e-5 

    A = (5.15e-17) #[Pa^-3][y^-1] warm ice

    #Calculate dissipations
    D0 = (b0 * (Q_0)**2 / (w_0 * h_0**2) + 
                m0 * ((Q_0)**(4/3) * h_0**(2/3) / (A**(1/3) * w_0**(8/3))))
    D1 = (b1 * (Q_1)**2 / (w_1 * h_1**2) + 
                m1 * ((Q_1)**(4/3) * h_1**(2/3) / (A**(1/3) * w_1**(8/3))))
    D2 = (b2 * (Q_2)**2 / (w_2 * h_2**2) + 
                m2 * ((Q_2)**(4/3) * h_2**(2/3) / (A**(1/3) * w_2**(8/3))))


    #Find the minimum dissipation point
    min_r = float('inf')
    min_x = 0.

    def dissipation(x):
        yb = 100000.
        r = D0 * np.sqrt((x - x0)**2 + (yb - y0)**2) + D1 * np.sqrt((x - x1)**2 + (yb - y1)**2) + D2 * np.sqrt((x - x2)**2 + (yb - y2)**2)
        return r

    out = minimize_scalar(dissipation, bounds = (0, 30000), method = 'bounded')
    min_x = out.x
    xListA.append(min_x)

In [ ]:
#Plot Figure 3b

fig, ax = plt.subplots(1,2, figsize = (10, 3.5))


ax[0].plot(greenland['k_ratio'], greenland['x_position']-1500, 'v', markersize = 9, color = '#44AA99', label = 'Elmer/Ice $w_0/h_0 = 6$')
ax[0].plot(np.linspace(0.2 * 5e-3, 3 * 5e-3, 100)/5e-3, (np.array(xListG)-1500), linestyle = 'dashdot', linewidth = 2, color = '#44AA99', label = "Fermat Prediction")

ax[1].plot(antarctica['k_ratio'], antarctica['x_position']-15000, 's', markersize = 8, color = '#332288', label = 'Elmer/Ice $w_0/h_0 = 60$')
ax[1].plot(np.linspace(0.2 * 4e-5, 4 * 4e-5, 100)/4e-5, (np.array(xListA)-15000), linestyle = 'dashed', linewidth = 2, color = '#332288', label = 'Fermat Prediction')


ax[0].set_ylabel('Bifurcation $x$-coordinate (m)', fontsize = 12)
ax[0].legend(loc = [0.02, 0.07], fontsize = 9.5)
#ax[0].axhline(1500, color = 'k', alpha = 0.25, linestyle = 'dashdot', zorder = 0)
#ax[0].axhline(-1500, color = 'k', alpha = 0.25, linestyle = 'dashdot', zorder = 0)
#plt.ylim([-16000, 16000])

ax[0].set_xlabel('$\\mu_1/\\mu_2$', fontsize = 12)
ax[1].set_xlabel('$\\beta_1/\\beta_2$', fontsize = 12)
ax[0].set_xscale('log')
ax[1].set_xscale('log')
ax[0].set_xlim([1.5e-1,6])
ax[1].set_xlim([1.5e-1,6])

#plt.axhline(-15000, color = 'k', alpha = 0.25, linestyle = 'dashed', zorder = 0)
#plt.axhline(15000, color = 'k', alpha = 0.25, linestyle = 'dashed', zorder = 0)


ax[1].legend(loc = [0.02, 0.07], fontsize = 9.5)
sns.despine()
ax[0].annotate('(a)', xy = (-0.2, 1), xycoords = "axes fraction", fontsize = 14)
ax[1].annotate('(b)', xy = (-0.2, 1), xycoords = "axes fraction", fontsize = 14)

plt.savefig('fig-3_elmer-greenland-detail.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
#Plot Figure 3c

m1, b1, r1, p1, se1 = linregress((np.array(xListG)-1500), greenland['x_position']-1500)
print(m1, r1**2, p1)

m2, b2, r2, p2, se2 = linregress((np.array(xListA)-15000), antarctica['x_position']-15000)
print(m2, r2**2, p2)

norm1 = mcolors.Normalize(vmin = greenland['k_ratio'].min(), vmax = greenland['k_ratio'].max())
norm2 = mcolors.Normalize(vmin = antarctica['k_ratio'].min(), vmax = antarctica['k_ratio'].max())

fig, ax = plt.subplots(figsize = (5.5, 3.5))

sc1 = ax.scatter((np.array(xListG)-1500)/1500, (greenland['x_position']-1500)/1500, c = greenland['k_ratio'], cmap = 'plasma_r', norm = norm1)
xtmp1 = np.linspace(-1500, 1500, 10)
ax.plot((xtmp1)/1500, (m1*xtmp1+b1)/1500, color = '#44AA99', linewidth = 3, linestyle = 'dashdot')


sc2 = ax.scatter((np.array(xListA)-15000)/15000, (antarctica['x_position']-15000)/15000, c = antarctica['k_ratio'], cmap = 'plasma_r', norm = norm2, marker = 's')
xtmp2 = np.linspace(-15000, 0.65*15000, 10)
ax.plot((xtmp2)/15000, (m2*xtmp2+b2)/15000, color = '#332288', linewidth = 3, linestyle = 'dashed')

ax.set_xlabel('Fermat Prediction ($\\frac{x_b}{w_0}$)', fontsize = 12)
ax.set_ylabel('Elmer Bifurcation Point ($\\frac{x_b}{w_0}$)', fontsize = 12)

ax.annotate("'Greenland' $w/h \\sim 6$", xy = (0.05, 0.79), xycoords = "axes fraction", fontsize = 10, color = '#44AA99')
ax.annotate("$X_{Elmer} \\propto 0.5 X_{Fermat}$", xy = (0.05, 0.7), xycoords = "axes fraction", fontsize = 14, color = '#44AA99')
ax.annotate("$R^2 = 0.98$", xy = (0.075, 0.61), xycoords = "axes fraction", fontsize = 10, color = '#44AA99')

ax.annotate("'Antarctic' $w/h \\sim 60$", xy = (0.35, 0.28), xycoords = "axes fraction", fontsize = 10, color = '#332288')
ax.annotate("$X_{Elmer} \\propto 0.9 X_{Fermat}$", xy = (0.35, 0.2), xycoords = "axes fraction", fontsize = 14, color = '#332288')
ax.annotate("$R^2 = 0.99$", xy = (0.37, 0.11), xycoords = "axes fraction", fontsize = 10, color = '#332288')

cbar = fig.colorbar(sc1, ax=ax, label='$\\frac{\\mu_1}{\\mu_2}$')
cbar.set_label('$\\frac{\\mu_1}{\\mu_2}$\n$\\frac{\\beta_1}{\\beta_2}$', fontsize=16, rotation = 0, labelpad = 30, y = 0.6)

cbar.set_ticks(greenland['k_ratio'])

ax.annotate('(c)', xy = (-0.2, 1), xycoords = "axes fraction", fontsize = 14)

sns.despine()
plt.axis('equal')
plt.savefig('fig-3_elmer-vs-fermat.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
xListG

In [ ]:
greenland